# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.020 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 29.809 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 33.180 MiB RAM in 0.094s, peaked 0.000 MiB above current, total RAM usage 62.988 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

100 loops, best of 3: 6.9 ms per loop
In [3] used 0.066 MiB RAM in 2.879s, peaked 0.000 MiB above current, total RAM usage 63.055 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [4]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 145.0)
In [4] used -1.973 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 61.082 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## Containers using the array module in Python

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
%load_ext memory_profiler

In [5] used 0.008 MiB RAM in 0.013s, peaked 0.000 MiB above current, total RAM usage 61.090 MiB


In [6]:
%memit arr = array.array('l', a)

peak memory: 76.36 MiB, increment: 15.26 MiB
In [6] used 15.449 MiB RAM in 0.259s, peaked 0.000 MiB above current, total RAM usage 76.539 MiB


7.7 ~ 15 MB vs 31 MB seems like a good deal, although sometimes Python is allocating more memory than necessary.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [7]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

16.19968

In [7] used 0.055 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 76.594 MiB


But how it performs during reductions?

In [8]:
t = %timeit -o sum(arr)

100 loops, best of 3: 11.5 ms per loop
In [8] used 0.051 MiB RAM in 4.755s, peaked 0.000 MiB above current, total RAM usage 76.645 MiB


In [9]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 86.7)
In [9] used 0.004 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 76.648 MiB


Well, that's a bit disappointing, as the array object performs about 2x slower than a regular list.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

## NumPy

In [10]:
import numpy as np

In [10] used 6.621 MiB RAM in 0.041s, peaked 0.000 MiB above current, total RAM usage 83.270 MiB


In [11]:
na = np.array(a, dtype=np.int64)

In [11] used 7.668 MiB RAM in 0.064s, peaked 0.000 MiB above current, total RAM usage 90.938 MiB


We see that, with 8 bytes/element, NumPy is also an efficient container (much more than the `array` package in standard library which takes ~16 bytes/element).

In [12]:
t = %timeit -o sum(na)

10 loops, best of 3: 89 ms per loop
In [12] used 0.020 MiB RAM in 3.651s, peaked 0.000 MiB above current, total RAM usage 90.957 MiB


In [13]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 11.241)
In [13] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 90.957 MiB


Oops, this is more than several times slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [14]:
t = %timeit -o na.sum()

1000 loops, best of 3: 551 µs per loop
In [14] used 0.121 MiB RAM in 2.285s, peaked 0.000 MiB above current, total RAM usage 91.078 MiB


In [15]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 1814.714)
In [15] used 0.008 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 91.086 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [16]:
import bcolz

In [16] used 22.207 MiB RAM in 0.174s, peaked 0.000 MiB above current, total RAM usage 113.293 MiB


In [17]:
ca = bcolz.carray(na)

In [17] used 0.691 MiB RAM in 0.005s, peaked 0.000 MiB above current, total RAM usage 113.984 MiB


In [18]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.69140625)
In [18] used 0.020 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 114.004 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [19]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [19] used 0.066 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 114.070 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [20]:
t = %timeit -o ca.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 1.87 ms per loop
('MIPS:', 533.584)
In [20] used 0.078 MiB RAM in 0.838s, peaked 0.000 MiB above current, total RAM usage 114.148 MiB


This is around 2~4x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 10x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [21]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [21] used 14.859 MiB RAM in 0.056s, peaked 0.000 MiB above current, total RAM usage 129.008 MiB


In [22]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [22] used 0.008 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 129.016 MiB


In [23]:
t = %timeit -o cau.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

1000 loops, best of 3: 1.93 ms per loop
('MIPS:', 517.007)
In [23] used 0.012 MiB RAM in 8.307s, peaked 0.000 MiB above current, total RAM usage 129.027 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the only source of the overhead (and it is very minor in fact).  The actual source of the difference is the memory layout of the different containers (bcolz layout is a bit more complex than NumPy).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be competitive with NumPy, performance wise, in some scenarios.

### Exercise: Using bcolz in real scenarios

bcolz does not get good compression ratios only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics:

* Which are the typical compression ratios for this case?

* Is there a difference in speed accessing data in compressed and non-compressed state (clevel=0)

* Which are the compressors achieving the best compression/speed ratio?